In [1]:
from bs4 import BeautifulSoup as bs
import requests 
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/jovanycandelario/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [6]:
mars_url = 'https://mars.nasa.gov/news/'

browser.visit(mars_url)

html = browser.html

mars_soup = bs(html, 'html.parser')

In [7]:
news_title = mars_soup.find_all('div', class_='content_title')[0].text
news_p = mars_soup.find_all('div', class_='article_teaser_body')[0].text

In [8]:
jpg_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(jpg_url)

html = browser.html

jpg_soup = bs(html, 'html.parser')

In [9]:
url_string = jpg_soup.find_all('img')[3]['src']

In [10]:
featured_image_url = jpg_url + url_string
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA24189-640x350.jpg'

In [11]:
table_url = 'https://space-facts.com/mars/'

In [12]:
tables = pd.read_html(table_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [18]:
df = tables[0]

html_table = df.to_html('table.html')

In [19]:
photos_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(photos_url)

html = browser.html

photos_soup = bs(html, 'html.parser')

In [57]:
all_hemispheres = photos_soup('div', class_='collapsible results')
hemisphere_titles = all_hemispheres.find_all('div', class_='item')

for n in hemisphere_titles:
    hem = n.find('div', class_='description')
    title = hem.h3.text

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

<h3>Cerberus Hemisphere Enhanced</h3>

In [64]:
conn ='mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

[]

In [ ]:
db = client.mars_db
collection = db.rover_news